In [1]:
# import statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import warnings
warnings.filterwarnings("ignore")


# function detect_face : process faces and converts to grayscale
def detect_face(img):
    # convert image to gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # load OpenCV face detector
    casc_path = "/Users/gelo/Desktop/Main/Study/CSS581/Project/hapn_code/data/opencv-files/haarcascade_frontalface_default.xml"
    face_cas = cv2.CascadeClassifier(casc_path)
    faces = face_cas.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=4);
    
    
    #if no faces
    if (len(faces) == 0):
        return None, None
    
    #extract face
    (x, y, w, h) = faces[0]
    #return just face
    return gray[y:y+w, x:x+h], faces[0]

# function prepare_training_data : handles logistics and setup
def prepare_training_data(data_folder_path):
    # get directories
    dirs = os.listdir(data_folder_path)
    
    # set up arrays for faces and labels
    faces = []  # is this a face?
    labels = [] # whose face is this?
    count = 0

    
    # loop through all folders with images
    for dir_name in dirs:
        # skip any hidden directories
        if dir_name.startswith("."):
            continue;
        # get path for each person folder
        path = os.path.join(data_folder_path, dir_name)
        # set the label to the folder name
        label = dirs.index(dir_name)
        
        # read each image, get and store faces
        for image_name in os.listdir(path):
            # ignore system files
            if image_name.startswith("."): 
                continue;
            # build image path
            image_path = os.path.join(path,image_name)
            # read image
            image = cv2.imread(image_path)
            # display image window showing image
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)
            # detect face
            face, rect = detect_face(image)
            
            # ignore faces not detected
            if face is not None:
                # store face
                faces.append(face)
                # add label
                labels.append(label)
            count+=1

                
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels, count

In [2]:
# data setup
faces, labels, count = prepare_training_data("data/people_faces")
print("Total faces: ", len(faces), "\nTotal labels: ", len(labels), "\nCount: ", count)

Total faces:  10 
Total labels:  10 
Count:  96


In [3]:
#create our LBPH face recognizer 
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
#train our face recognizer of our training faces
# face_recognizer.train(faces, np.array(labels))